In [1]:
import torch
torch.cuda.empty_cache()
from datasets import load_dataset
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification,\
    TrainingArguments, Trainer, pipeline, DataCollatorWithPadding, set_seed
import evaluate
import numpy as np
import pandas as pd
import torch.nn as nn
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
set_seed(42)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
#Get the dataset
snli = load_dataset("snli")

Found cached dataset snli (C:/Users/zebzi/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# per chatGPT's suggestion, let's concatenate premise + hypothesis into a "text" column.
train_text = np.char.add(np.char.add(snli['train']['premise'], ' '), snli['train']['hypothesis'])
test_text = np.char.add(np.char.add(snli['test']['premise'], ' '), snli['test']['hypothesis'])
valid_text = np.char.add(np.char.add(snli['validation']['premise'], ' '), snli['validation']['hypothesis'])

# let's do a sanity check to make sure concatenation was done correctly
print(f'--train_text[0]--: {train_text[0]}')
print(f'--original + hypothesis--: {snli["train"]["premise"][0]+snli["train"]["hypothesis"][0]}\n')

print(f'--test_text[0]--: {test_text[0]}')
print(f'--original + hypothesis--: {snli["test"]["premise"][0]+snli["test"]["hypothesis"][0]}\n')

print(f'--valid_text[0]--: {valid_text[0]}')
print(f'--original + hypothesis--: {snli["validation"]["premise"][0]+snli["validation"]["hypothesis"][0]}')

snli["train"] = snli["train"].remove_columns(["premise", "hypothesis"])
snli["train"] = snli["train"].add_column("text", train_text)

snli["test"] = snli["test"].remove_columns(["premise", "hypothesis"])
snli["test"] = snli["test"].add_column("text", test_text)

snli["validation"] = snli["validation"].remove_columns(["premise", "hypothesis"])
snli["validation"] = snli["validation"].add_column("text", valid_text)

--train_text[0]--: A person on a horse jumps over a broken down airplane. A person is training his horse for a competition.
--original + hypothesis--: A person on a horse jumps over a broken down airplane.A person is training his horse for a competition.

--test_text[0]--: This church choir sings to the masses as they sing joyous songs from the book at a church. The church has cracks in the ceiling.
--original + hypothesis--: This church choir sings to the masses as they sing joyous songs from the book at a church.The church has cracks in the ceiling.

--valid_text[0]--: Two women are embracing while holding to go packages. The sisters are hugging goodbye while holding to go packages after just eating lunch.
--original + hypothesis--: Two women are embracing while holding to go packages.The sisters are hugging goodbye while holding to go packages after just eating lunch.


In [5]:
snli = snli.filter(lambda example: example["label"] != -1)
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
    
tokenized_snli= snli.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-a7e3f6a12dd5470b.arrow
Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-10263cd2a97d376b.arrow
Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-7dd2e0b9c4e30d43.arrow
Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-6a07a2c8d2f1ad8c.arrow
Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-aa9d2d1182cd7304.arrow
Loading cached processed 

In [6]:
# Load the model that was trained in HW1
id2label = {0: "entailment", 1: "neutral", 2:"contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

config = AutoConfig.from_pretrained("config.json")
model = AutoModelForSequenceClassification.from_pretrained("pytorch_model.bin", config=config)

OSError: config.json is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
# Load the pipeline for natural language inference tasks
print(f"Model type: {type(model)}")
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
print(f"pipeline type: {type(nlp)}")
tokenized_snli["test"]["text"][0]
count = 0
i = 0

rowsList = []
# while(count < 50):
#     try:
#         temp = tokenized_snli["test"]["text"][i + 100].split(".")
#         # print(tokenized_snli["test"]["text"][i + 100])
#         premise = temp[0]
#         hypothesis = temp[1]
#     except:
#         print("no period\n")

#     result = nlp(tokenized_snli["test"]["text"][i + 100])[0]
#     temp = []
#     if(result["label"] != id2label[tokenized_snli["test"]["label"][i+100]]):
#         count += 1
#         # print()
#         # print("Premise: ",end="")
#         # print(premise)
#         # print("Hypothesis: ",end="")
#         # print(hypothesis)
#         # print()
#         # print(f"Confidence: {result['score']}")
#         # print()
#         # print("The prediction is: ", result["label"])
#         # print("The correct is: ", id2label[tokenized_snli["test"]["label"][i+100]])
#         temp = [str(count), f"Premise: {premise} \nHypothesis: {hypothesis}", id2label[tokenized_snli["test"]["label"][i+100]], f"Confidence: {result['score']:.3f} \n" + result['label'], str(len(premise) + len(hypothesis))]
#         rowsList.append(temp)
#     i += 1

# with open("hw5CSV.csv", mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["", "Input Text", "Ground-Truth Label", "Predicted Label", "Input Text Length"])
#     writer.writerows(rowsList)
# print("Here is i: ", i)

c:\Users\zebzi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\pipelines\base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Here is i:  459
